# Trabalho Final GCC108 - Teoria da Computação #

Prof.: Douglas H. S. Abreu

Nomes:
- Iago Manoel Brito de Sá Vaz da Silva
- Vinicius Caputo de Castro

Turma: 10A

Repositório: <https://github.com/iagocq/gcc108-trabalho-final>

---

# Resolução dos Exercícios

#### 1) Descreva com suas palavras uma estratégia para o desenvolvimento de uma máquina de Turing que compute a soma de 2 números binários.

No papel, a soma de números binários é realizada dígito a dígito, da direita à esquerda. No entanto, isso é um pouco inconveniente no contexto de máquinas de Turing, visto que a fita é limitada à esquerda, e na soma os resultado cresce nessa direção. Para combater isso, a máquina soma os dígitos dos argumentos na ordem natural, mas escreve o resultado logo após o segundo argumento, na ordem reversa.

Para a soma em si, o processo é o seguinte:

- Soma um par de dígitos ainda não processado e escreve o resultado. Se essa soma resulta em um carry (vai um, $1+1 = 0$ e um carry), 2 é escrito ao invés disso.

- Quando um outro par de dígitos é somado e 2 é encontrado no final do resultado, outra soma é realizada com 1. Se o resultado é maior ou igual a 10, outro carry é colocado na frente do resultado.

Para apresentar o resultado, os dois argumentos iniciais são apagados e o resultado é escrito de trás pra frente no início da fita.


#### 2) Faça o esboço por meio de desenho da máquina de Turing proposta.
![](Img/soma.png)

#### 3) Defina a MT como uma quíntupla $M=(Q, \Sigma, \Gamma, \delta, q_0)$

In [102]:
# Transforma arquivo jff de uma máquina em sua representação formal

import xml.etree.ElementTree as ET

arquivo = 'soma.jff'
automato = ET.parse(arquivo).getroot().find('automaton')

# conjunto de estados
Q = {}
# alfabeto de entrada
sigma = ['0', '1']
# alfabeto da fita
gamma = set(sigma)
# função de transição
delta = []
# estado inicial
q0 = None

for estado in automato.findall('state'):
    id = estado.get('id')
    nome = estado.get('name')
    Q[id] = nome
    if estado.find('initial') is not None:
        if q0 is not None:
            raise ValueError('Existe mais de um estado inicial')
        q0 = id

for transicao in automato.findall('transition'):
    q_i = transicao.findtext('from')
    x = transicao.findtext('read')
    q_j = transicao.findtext('to')
    y = transicao.findtext('write')
    D = transicao.findtext('move')
    x = x if x else 'B'
    y = y if y else 'B'
    delta.append(((q_i, x), (q_j, y, D)))
    gamma.add(x)
    gamma.add(y)

delta.sort(key=lambda k: int(Q[k[0][0]][1:]))


In [101]:
from IPython.display import display, Math

texto = f'M = (Q, \Sigma, \Gamma, \delta, q_0) \\\\'

estados = Q.values()
texto += 'Q = \{\,'
for i, estado in enumerate(estados):
    texto += estado
    if i < len(estados)-1:
        texto += ',\ '
    if i % 13 == 12:
        texto += ' \\\\ '

texto += '\,\} \\\\'

texto += f'\Sigma = \{{\, {", ".join(sigma)} \,\}} \\\\'
texto += f'\Gamma = \{{\, {", ".join(gamma)} \,\}} \\\\'

for i, transicao in enumerate(delta):
    (q_i, x), (q_j, y, D) = transicao
    texto += f'\delta({Q[q_i]}, {x}) = ({Q[q_j]}, {y}, {D})'
    if i < len(delta)-1:
        texto += ',\ '
    if i % 3 == 2:
        texto += '\\\\'

texto += f'\\\\ q_0 = {Q[q0]}'

display(Math(texto))

<IPython.core.display.Math object>

#### 4) Faça a conversão de $M$ em $R(M)$

In [105]:
gamma_cod = {g: '1'*(i+1) for i, g in enumerate(gamma)}
Q_cod = {q: '1'*(i+1) for i, q in enumerate(Q)}
D_cod = {'L': '1', 'R': '11'}

r = '000'

for i, transicao in enumerate(delta):
    (q_i, x), (q_j, y, D) = transicao
    r += Q_cod[q_i] + '0' + gamma_cod[x] + '0' + Q_cod[q_j] + '0' + gamma_cod[y] + '0' + D_cod[D]
    r += '00'
r += '0'

print('R(M) = ' + r)

R(M) = 000101111101101111101100110110111011010011011111011101111101001101110110111011001101011010110011101110111101101100111011111011111111111111110111110110011101011111011011001111011011110110110011110111110111111011111011001111101111101111111011111011001111101101111101101100111111010111111010110011111101110111111011101100111111011011111111011010011111101111101111111101111101001111111010111111101011001111111011101111111011101100111111101101111111110110100111111101111101111111110111110100111111110101111111111011011001111111101111101111111111011111011001111111101110111111111110110110011111111101111101111111111110111110110011111111101011111111111101101100111111111011101111111111011011001111111111011111011111111111111011111011001111111111011011111111110110110011111111111011011111111111011011001111111111101111101111111111111011111011001111111111110110111111111111011011001111111111110111110111111111111111011111011001111111111111011110111111111111111110111011001111111111111011101111111111111

#### 5) Desenvolva uma função MTU que receba $R(M)$ acrescido de uma entrada $w$

In [150]:
class Celula:
    def __init__(self, L=None, R=None, v='B'):
        self.L = L
        self.R = R
        self.v = v

class Fita:
    def __init__(self):
        self.c = Celula()

    def L(self):
        if self.c.L is None:
            self.c.L = Celula(R=self.c)
        self.c = self.c.L

    def R(self):
        if self.c.R is None:
            self.c.R = Celula(L=self.c)
        self.c = self.c.R

    def ler(self):
        return self.c.v

    def escrever(self, v):
        self.c.v = v

def MTU(r: str, w: str):
    r = r.split('000')[1]
    w = 'B' + 'B'.join(open(w).read().strip().split(';'))
    fita = Fita()
    for v in w:
        fita.escrever(v)
        fita.R()

    for _ in range(2):
        fita.L()
        while fita.ler() != 'B':
            fita.L()

    gamma_decod = {v: k for k, v in gamma_cod.items()}
    D_decod = {v: k for k, v in D_cod.items()}

    delta = {}

    for transicao in r.split('00'):
        q_i, x, q_j, y, D = transicao.split('0')
        delta[q_i, gamma_decod[x]] = q_j, gamma_decod[y], D_decod[D]

    q = Q_cod[q0]
    while (q, fita.ler()) in delta:
        q, y, D = delta[q, fita.ler()]
        fita.escrever(y)
        if D == 'L':
            fita.L()
        else:
            fita.R()

    fita.R()
    res = ''
    while fita.ler() != 'B':
        res += fita.ler()
        fita.R()
    return res

MTU(r, 'exemplo2.CSV')

'1100101'

#### 6-A) Explique a tese de Church-Turing de forma sucinta.

As máquinas de Turing são uma formalização de um procedimento efetivo, logo se uma função é computada por uma máquina de turing ela é efetivamente computável.

f é computavel por uma máquina de turing $\Longrightarrow$ f é efetivamente computável

A tese de Church-Turing diz: toda "função naturalmente computável" pode ser computada por uma máquina de Turing, então a tese faz a "volta" da primeira afirmação.

f é computavel por uma maquina de turing $\iff$ f é efetivamente computavel


#### 6-B) Dada uma máquina de Turing arbitrária $M$ e uma string de entrada $w$, a computação de $M$ com entrada $w$ irá parar em menos de 100 transições? Descreva uma máquina de Turing que resolva esse problema de decisão.

Uma máquina de Turing universal recebe a máquina $M$ como $R(M)$ e a executa com entrada $w$, porém limita sua execução a 100 transições. Caso o limite de execução seja ultrapassado, 0 é escrito na fita, senão 1 é escrito.

#### 6-C) Mostre a solução para cada um dos seguintes sistemas de correspoondência de Post

a) (a, aa), (bb, b), (a, bb)

aabbbb

| 1   | 3     | 2     | 2      |
| --- | ----- | ----- | ------ |
| a   | aa    | aabb  | aabbbb |   
| aa  | aabb  | aabbb | aabbbb |

b) (a, ab), (ba, aba), (b, aba), (bba, b)

abbaba

| 1   | 4     | 2      |
| --- | ----- | ------ |
| a   | abba  | abbaba |
| ab  | abb   | abbaba |

c) (abb, ab), (aba, ba), (aab, abab)

| 1   | 2     | 2         | 1             |
| --- | ---   | ---       | ---           |
| abb | abbaba| abbabaaba | abbabaabaabb  |
| ab  | abba  | abbaba    | abbabaab      |

Como nenhuma peça da parte de baixo começa com aa, não existe solução.

d) (ab,aba), (baa, aa), (aba, baa)

| 1   | 3     | 3         | 3            |
| --- | ---   | ---       | ---          |
| ab  | ababa | ababaaba  | ababaabaaba  |
| aba | ababaa| ababaabaa | ababaabaabaa |

Como a peça 3 é a unica possivel para a sequência ela será a unica usada, mas a palavra nunca chegará ao fim, pois sempre haverá a diferença de 1 caractere entre a palavra 1 e a palavra 2. Logo a correspondencia não tem solução.

e) (a, aaa), (aab, b), (abaaa, ab)

aaab

| 1   | 2    |
| --- | ---  |
| a   | aaab |
| aaa | aaab |


f) (ab, bb), (aa, ba), (ab, abb), (bb, bab)

| 3   | 4     | 3         | 1             | 4             | 4                 |
| --- | ---   | ---       | ---           | ---           | ---               |
| ab  | abbb  | abbbab    | abbbabab      | abbbababbb    | abbbababbbbb      |
| abb | abbbab| abbbababb | abbbababbbb   | abbbababbbbbab| abbbababbbbbabbab |

Como não tem nenhuma peça que seja menor do lado 2 do que do lado 1, a palavra 1 nunca chegará a um fim a partir desse ponto. Logo a correspondencia não tem solução.